[View in Colaboratory](https://colab.research.google.com/github/kyokicchi/kaggle_codes/blob/master/trainLGBmodel.ipynb)

In [1]:
!pip install lightgbm

    100% |████████████████████████████████| 1.1MB 15.9MB/s 


In [0]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from google.colab import drive
from google.colab import files
from sklearn.model_selection import train_test_split
import pickle

In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%cd ../content/gdrive/My Drive/projects/
%ls

/content/gdrive/My Drive/projects
model_LGB.sav  model_XGB.sav  user_test_n.csv
model_NN.h5    output.csv     user_train_n.csv


In [0]:
s_data = 'user_train_n.csv'
filename = 'model_LGB.sav'
s_col_tgt = 'transactionRevenue_sum'
l_col_drop = [s_col_tgt]

In [6]:
%%time
df_train = pd.read_csv(s_data, index_col = 0)

<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 9.17 s, sys: 896 ms, total: 10.1 s
Wall time: 13.5 s


In [7]:
df_train.columns

Index(['visitNumber_sum', 'visitNumber_min', 'visitNumber_max',
       'visitNumber_mean', 'visitStartTime_min', 'visitStartTime_max',
       'visitStartTime_mean', 'operatingSystem_min', 'operatingSystem_max',
       'operatingSystem_mean', 'country_mean', 'subContinent_mean', 'hits_sum',
       'hits_min', 'hits_max', 'hits_mean', 'pageviews_sum', 'pageviews_min',
       'pageviews_max', 'pageviews_mean', 'transactionRevenue_sum',
       'transactionRevenue_size', 'source_mean', 'month_min', 'month_max',
       'month_mean', 'day_min', 'day_max', 'day_mean', 'weekday_mean',
       'weekofyear_min', 'weekofyear_max', 'weekofyear_mean', 'weekend_mean',
       'browser_cat_mean', 'ads_1_mean', 'ads_2_mean',
       'user_cumcnt_per_day_sum', 'user_cumcnt_per_day_min',
       'user_cumcnt_per_day_max', 'user_cumcnt_per_day_mean',
       'user_sum_per_day_sum', 'user_sum_per_day_min', 'user_sum_per_day_max',
       'user_sum_per_day_mean', 'user_cumcnt_sum_ratio_per_day_mean',
       'Affi

In [8]:
%%time
df_y = df_train[s_col_tgt]
df_x = df_train.drop(l_col_drop, axis=1)

CPU times: user 59 ms, sys: 178 ms, total: 237 ms
Wall time: 237 ms


In [9]:
%%time
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y)
x_eval, x_valid, y_eval, y_valid = train_test_split(x_test, y_test)



CPU times: user 1.05 s, sys: 123 ms, total: 1.18 s
Wall time: 1.18 s


In [0]:

params = {
    'learning_rate': 0.03,
    'n_estimators': 200,
    'num_leaves': 128,
    'subsample': 0.2217,
    'colsample_bytree': 0.6810,
    'min_split_gain': np.power(10.0, -4.9380),
    'reg_alpha': np.power(10.0, -3.2454),
    'reg_lambda': np.power(10.0, -4.8571),
    'min_child_weight': np.power(10.0, 2),
    'silent': True
}

model = lgb.LGBMRegressor(**params)



In [11]:
%%time
model.fit(x_train, 
          y_train,
          eval_set=[(x_train, y_train),(x_eval, y_eval)],
          eval_metric='rmse',
          early_stopping_rounds=100)

[1]	training's rmse: 0.00487753	training's l2: 2.37903e-05	valid_1's rmse: 0.00492602	valid_1's l2: 2.42657e-05
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 0.00482719	training's l2: 2.33018e-05	valid_1's rmse: 0.00487395	valid_1's l2: 2.37554e-05
[3]	training's rmse: 0.00477656	training's l2: 2.28155e-05	valid_1's rmse: 0.0048226	valid_1's l2: 2.32575e-05
[4]	training's rmse: 0.0047301	training's l2: 2.23738e-05	valid_1's rmse: 0.00477578	valid_1's l2: 2.28081e-05
[5]	training's rmse: 0.00468431	training's l2: 2.19428e-05	valid_1's rmse: 0.00472888	valid_1's l2: 2.23623e-05
[6]	training's rmse: 0.00464055	training's l2: 2.15347e-05	valid_1's rmse: 0.00468408	valid_1's l2: 2.19406e-05
[7]	training's rmse: 0.00459884	training's l2: 2.11493e-05	valid_1's rmse: 0.00464083	valid_1's l2: 2.15373e-05
[8]	training's rmse: 0.0045595	training's l2: 2.0789e-05	valid_1's rmse: 0.00460078	valid_1's l2: 2.11672e-05
[9]	training's rmse: 0.00452101	training's l2

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.681,
       importance_type='split', learning_rate=0.03, max_depth=-1,
       min_child_samples=20, min_child_weight=100.0,
       min_split_gain=1.153453257821093e-05, n_estimators=200, n_jobs=-1,
       num_leaves=128, objective=None, random_state=None,
       reg_alpha=0.0005683292391383383, reg_lambda=1.3896326196366016e-05,
       silent=True, subsample=0.2217, subsample_for_bin=200000,
       subsample_freq=0)

In [12]:
%%time
pickle.dump(model, open(filename, 'wb'))

# files.download(filename) 

CPU times: user 221 ms, sys: 3.01 ms, total: 224 ms
Wall time: 605 ms
